In [17]:
import pandas as pd

price_data = pd.read_csv("stock_data_from_2015.csv")
live_price = pd.read_csv("live_price_2020-11-19.csv")
EBIT = pd.read_csv("EBIT_data.csv")
AT = pd.read_csv("AT_data.csv")
shares_outstanding = pd.read_csv("shares_outstanding.csv")

print(len(price_data), len(live_price), len(EBIT), len(AT), len(shares_outstanding))

ticker_unique = EBIT.ticker.unique()
latest_EBIT = pd.DataFrame()

for ticker in ticker_unique:
    ticker_df = EBIT[EBIT.ticker == ticker]
    ticker_df = ticker_df.sort_values(by = "endDate")
    ticker_df.reset_index(drop = True, inplace = True)
    latest_EBIT = latest_EBIT.append(ticker_df[-1:])
    #latest_AT = latest_AT.append(ticker_df[0])

latest_EBIT

latest_EBIT.to_csv("current_EBIT.csv")

## get the market value of a company

In [80]:
so_list = []
live_list = []
ticker_list = []

for ticker in shares_outstanding.ticker:
    try:
        df = live_price[live_price.ticker == ticker]
        live_list.append(df[0:1].price.item())
        ticker_list.append(ticker)
        so_list.append(shares_outstanding[shares_outstanding.ticker == ticker][0:1]["so"].item())
    except:
        pass


In [85]:
market_value = pd.DataFrame({
    "ticker" : ticker_list,
    "so" : so_list,
    "price" : live_list
})

In [87]:
market_value["mar_cap"] = market_value.so * market_value.price

In [89]:
market_value.to_csv("market_capitalization.csv")

### Add Ebit to df

In [113]:
ebit_list = []
ebit_ticker = []

for ticker in market_value.ticker:
    try:
        t_e = EBIT[EBIT.ticker == ticker]["EBIT"][0:1].item()
        ebit_list.append(t_e)
        ebit_ticker.append(ticker)
    except:
        pass

In [118]:
rel_ebit = pd.DataFrame({
    "ticker" : ebit_ticker,
    "ebit" : ebit_list
})

In [114]:
at_list = []
at_ticker = []

for ticker in market_value.ticker:
    try:
        t_e = AT[AT.ticker == ticker]["AT"][0:1].item()
        at_list.append(t_e)
        at_ticker.append(ticker)
    except:
        pass

In [117]:
rel_at = pd.DataFrame({
    "ticker" : at_ticker,
    "at" : at_list
})

In [116]:
missing = []

for i in ebit_ticker:
    if i in at_ticker:
        pass
    else:
        missing.append(i)
        
len(missing)

9

Drop the rows with tickers that are not in AT

In [124]:
missing_in_at = rel_ebit[rel_ebit.ticker.isin(missing)]
rel_ebit_2 = rel_ebit.drop(missing_in_at.index)

In [128]:
rel_ebit_2

,ticker,ebit
0,DDD,-7446000
1,MMM,1928000000
2,WBAI,-52337000
3,EGHT,-33098000
4,AHC,-1194000
...,...,...
5267,WYY,1148816
5268,WTT,-333000
5269,XTNT,760000
5270,ZDGE,390000


In [149]:
ebit_list = []

for ticker, at in zip(rel_at["ticker"], rel_at["at"]):
    try:
        ebit_list.append(rel_ebit[rel_ebit.ticker == ticker][0:1]["ebit"].item())
    except:
        ebit_list.append("")

rel_at["ebit"] = ebit_list

In [158]:
mar_cap_list = []

for ticker, at in zip(rel_at["ticker"], rel_at["at"]):
    try:
        mar_cap_list.append(market_value[market_value.ticker == ticker][0:1]["mar_cap"].item())
    except:
        mar_cap_list.append("")

rel_at["mar_cap"] = mar_cap_list

In [168]:
rel_at.ebit = pd.to_numeric(rel_at.ebit)

Drop missing values

In [171]:
rel_price_data = price_data[["adjclose", "ticker"]]

In [173]:
rel_at.to_csv("relevant_fundamentals.csv")
rel_price_data.to_csv("relevant_price_data.csv")